# Tanzanian Water Wells

In [103]:
import numpy as np
import pandas as pd
# settings to display all columns
pd.set_option("display.max_columns", None)
import matplotlib.pyplot as plt
import seaborn as sns

## Project Overview
Our group has been hired by Tanzania's Ministry of Water to develop a classification model that will assist Tanzania's Ministry of Water in predicting the status of the waterpoints they oversee. Although the waterpoint statuses could be described in a number of ways - e.g. whether it is functional but needs repair - our model will perform a binary classification to determine if the waterpoint is functioning as exepected or not.


## Business Problem
While the country of Tanzania has abundant water resources including Lake Victoria, the largest lake in Africa, nearly 50% of the waterpoints overseen by the Ministry of Water are non-functional, leaving large segments of the population without a reliable or potable source of water. The Ministry has decided to address this issue by having a predicitve model created that will allow them to identify the functionality of waterpoints. The model needed to take into consideration the fact that if wells were incorrectly identified as being functional then communities would be left without water, and conversely if the model identified working wells as broken the Ministry would not be able to allocate their resources effectively. Since this model is being used in a purely predictive context, its complexity is irrelevant -- it only matters how well it optimizes on these points. Since communities without water is the more tragic of the two considerations, we focused on optimizing for correctly identifying non-functional wells when they are indeed non-functional. This yielded our 'postive' target, being non-functional, as well as our metric, which is recall (true non-functional / (true non-functional + false functional)).

**Note:** 'well', 'pump', 'watersource', etc. etc. are all used interchangeably to refer to the target, which is its function status

## Data Understanding

This data comes to us from the major stakeholder -- the Tanzanian Ministry of Water. It is intended that our model be as accurate as possible for the data that we're given so that in the future, the Tanzanian government can use new data it has collected in order to predict the function state of a water pump for which it has data.

The data is a collection of information on each water pump, split into mostly descriptors and not much quantitative information.

### Columns

There are 40 columns in the dataset, not including our predictor feature 'status'. There are 31 categorical columns and 9 numeric columns. A description of the columns can be found [here](https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/page/25/#features_list).



In [104]:
# import well status and data information
status = pd.read_csv('data/status.csv', index_col='id')
data = pd.read_csv('data/data.csv', index_col='id')

In [105]:
# merge both on pump id
df = pd.merge(status, data, on='id')

In [106]:
# drop duplicates
df.drop_duplicates(inplace=True)
df.head()

,status_group,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
69572,functional,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
8776,functional,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
34310,functional,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
67743,non functional,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
19728,functional,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [107]:
# status counts
df['status_group'].value_counts()

functional                 32237
non functional             22813
functional needs repair     4314
Name: status_group, dtype: int64

In [ ]:
# visualize
fig, ax = plt.subplots()
sns.barplot(x=df['status_group'].value_counts().index, y=df['status_group'].value_counts(), ax=ax)
ax.set_title('Waterpoint Functionality')
ax.set_ylabel('Count')
plt.savefig('images/waterpoint_functionality_initial.png');

Based on the breakdown of the groups, we decided to turn our status column into a binary to make our classification process more straightforward. 

The 'functional needs repair' classification will be grouped in with the non-functional classification. The idea being, maintenance and repair will still be valuable on a damagaged waterpoint. It makes sense for our stakeholder to include in damaged waterpoints with this group because they are looking for where to direct their attention. It would be much more costly to the people of the area for a damaged waterpoint to be classified as completely functional and not get any additional attention (or water). 

In [ ]:
# combine 'functional needs repair' into 'non functional'
df.loc[df['status_group'] == 'functional needs repair', 'status_group'] = 'non functional'

After our status combination, our split shows a much more balanced distribution of our data.

In [ ]:
# Visualizing our breakdown

fig, ax = plt.subplots(figsize=(10,8))
plt.style.use('seaborn-talk')
palette = sns.color_palette("mako", n_colors = 2)
palette.reverse()
sns.barplot(x=df['status_group'].value_counts().index, y=df['status_group'].value_counts(),
           palette=palette)
ax.set_title('Waterpoint Functionality', fontsize=25)
ax.set_xticklabels(['Functional', 'Non-Functional'], fontsize=20)
ax.tick_params(axis='y', labelsize=20)
ax.set_ylabel('Count', fontsize=20)
plt.savefig('images/waterpoint_functionality.png');

We can now start diving into our numerical and categorical columns, as well as looking for null values and other problematic values in the dataset.

In [ ]:
# information on the full dataset
df.info()

In [ ]:
df.describe()

In [ ]:
# separate into numerical and categorical features
cat_cols = []
num_cols = []
for col in df.columns:
    if df[col].dtype in ['object']:
        cat_cols.append(col)
    else:
        num_cols.append(col)

We would like to get a better idea of the distribution of our numerical features.

In [ ]:
# visualize numericals
fig, axes = plt.subplots(nrows=len(num_cols),figsize=(20,30))
plt.tight_layout()
for i, col in enumerate(num_cols):
    sns.histplot(df[col], bins=50, ax=axes[i])

**Initial Numerical Column Impressions**

Very little of our numerical data can be classified as having a normal distribution. Because of this, we will avoid using a StandardScaler. We can see problems in our longitude data as it is not possible for there to be a longitude of 0 in Tanzania. We also see that there is a large proportion of 0 values in population and construction year that we will address later. 

It doesn't matter that `amount_tsh` has a large propportion of 0s. The data dictionary describes it as "amount_tsh - Total static head (amount water available to waterpoint)". This might seem quite bad, because this makes it sound like the total water available is 0 for over 70% of water points. However, this is an incorrect description for Total Static Head, as shown [here](../../notebooks/Nick/images/total_static_head.jpeg). It's simply the distance from the surface of the source to the surface of the access point, so actually having '0' is a good thing.

In [ ]:
# percent 0 in each column
(df == 0).sum(axis=0)/len(data)*100

After reviewing the zeros in each category, we noticed that there are 0 values in the longitude column. There are no longitudinal coordinates in Tanzania with a value of zero. We know these values are errors so we will replace them and the latidudinal coordinates associated with them with NAN. 

In [ ]:
# Number of rows with problem 0 longitude values 
(df['longitude'] == 0).sum()

In [ ]:
# All 0 longitude values have the same value count, showing us this is an error

df[df['longitude'] == 0]['latitude'].value_counts()

In [ ]:
# transform incorrect long/lat to nulls 
df.loc[(df['longitude']== 0), ('longitude','latitude')] = np.nan

We also will transform our contruction year with a value of 0 to NAN. We do not want this 0 value to have a negative impact on our model so replacing it is necessary. 

In [ ]:
# transform construction year 0 to nulls
df.loc[(df['construction_year']== 0), 'construction_year'] = np.nan

In [ ]:
# check out upper tail of amount_tsh
df['amount_tsh'].sort_values(ascending=False).head(30)

Although it is not indicated in our data description, we assume that the population is being measured in units of 100.

In [ ]:
# check out upper tail of population
df['population'].sort_values(ascending=False).head(30)

In [ ]:
# percent null in each column
df.isna().sum()/len(data)*100

Looking through the characteristics of unique values in our columns as well as the number of unique values in each column. This is important because it will inform how we move forward with preprocessing.

In [ ]:
#Looking through the characteristics of unique values in our columns
for col in df.columns:
    print(f'{col}:')
    print(df[col].value_counts(normalize=True))
    print(df[col].nunique())
    print('<>'*5)

In [ ]:
#Dialing in on the specifics of just our categorical columns
for col in cat_cols:
    print(f'{col}:')
    print(df[col].value_counts(normalize=True))
    print('<>'*5)

In [ ]:
# A quick look at our row value count, unique value count
# top category and freqency of that top category for each categorical feature 
df[cat_cols].describe()

In [ ]:
#Checking for hidden empty values
((df[cat_cols] == 'None')|(df[cat_cols] == 'none')).sum(axis=0)/len(data)*100

In [ ]:
((df[cat_cols] == 'Unknown')|(df[cat_cols] == 'unknown')).sum(axis=0)/len(data)*100

In [ ]:
df[cat_cols].isna().sum()

In [ ]:
(df[cat_cols] == 0).sum()

After spending time looking for a variety of representations of empty or null values in our categorical columns, we replace them all with Unknown for better categorization.

In [ ]:
df[cat_cols] = df[cat_cols].replace(to_replace=['none','None','unknown', 0, np.nan],
                                    value='Unknown')

In [ ]:
(df[cat_cols] == 'Unknown').sum()

We can now more accurately understand how unknown or empty values are represented in each feature across our dataset. 

As we explore the data, we will unpack some of the meaning behind the data and look for columns that are not relevant to our model. We are also now able to make decisions on dealing with the kept data. We used the data that was the most descriptive because our best-performing model downstream is able handle large amounts of categorical information. 

## Categorical feature determinations:

### Dropped Columns

The following columns will be dropped because they contain an exact duplicate of an existing column or data that is better represented in another column. 

- num_private
- region_code 
- district_code
- date_recorded 
- wpt_name
- recorded_by 
- scheme_name
- extraction_type_group
- extraction_type_class
- management_group
- payment_type
- quality_group
- quantity_group
- source_type
- source_class
- waterpoint_type_group

### OneHotEncoder Processing
The following columns have less than 25 unique values, making them good candidates for [OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html). The cutoff criteria was set to keep our post processing column count from getting too large relative to our number of rows in our dataset.

- basin
- region
- public_meeting
- scheme_management
- permit
- extraction_type
- management
- payment
- water_quality
- quantity
- source
- waterpoint_type

### Categorical Encoder Processing  

The following features have over 100 unique value counts, making them poor candidates for OneHotEncoder. To preserve the size of our post processing dataset we used [Count Encoding](https://contrib.scikit-learn.org/category_encoders/count.html). 

- funder 
- installer
- subvillage
- lga 
- ward

### Numerical Processing

Because our numerical data holds non-normal distributions, we used a [MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html) to standardize our data. We also used an [Iterative Imputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.IterativeImputer.html) to deal with the few null values we identified.

- amount_tsh
- gps_height
- longitude
- latitude
- population
- construction_year


In [ ]:
drop_cols = ['num_private', 'region_code', 'district_code', 'date_recorded', 'wpt_name',
             'recorded_by', 'scheme_name', 'extraction_type_group', 'extraction_type_class',
             'management_group', 'payment_type', 'quality_group', 'quantity_group',
             'source_type', 'source_class', 'waterpoint_type_group']
df_post = df.drop(columns=drop_cols)

We are making our status column binary for usability. 

In [ ]:
# binary encode status_group into `non_functional` with 1 = non_functional, 0 = functional
df_post.rename(columns={'status_group':'non_functional'}, inplace=True)
df_post['non_functional']\
= df_post['non_functional'].apply(lambda x: 1 if x == 'non functional' else 0)

In [ ]:
df_post.to_csv('data/data_cleaned.csv')

## Limitations

Overall, this data does a half-way serviceable job of presenting features that have relevance to the target functionality state. It's decently clean, and it presents geographical information, data about the well itself, the water that flows through it, and the groups involved in its operation. However, it's also not as ideal as it could be. We don't get a lot of information from the few numerical features, especially not as much as we would expect from this type of column. Furthermore, some categorical features like those relating to the water could probably be much more valuable if they were presented as quantifiable information. The huge proportion of categorical features in comparison to the numerical would suggest that the best model for this data would be non-parametric, as parametric models tend to perform better with a higher proportion of numerical features.

While our model might perform decently well with the provided data, I would strongly suggest that the Government of Tanzania revisit their data collection methods to provide more robust and predictive information.